In [15]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [16]:
!rm -rf logs

In [17]:

from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout,
    BatchNormalization,
    concatenate,
)
import tensorflow as tf
lamb = 0.0001
do = 0.35
lr = 0.0001


char_shape = 960
word_shape = 201
par_shape = 400
rest_shape = 27



n_weights = 300

char_model_input = Input(shape=(char_shape,))
char_model1 = BatchNormalization(axis=1)(char_model_input)
char_model2 = Dense(
    n_weights,
    activation=tf.nn.relu,
    kernel_regularizer=tf.keras.regularizers.l2(lamb),
)(char_model1)
char_model3 = Dropout(do)(char_model2)

char_model4 = Dense(
    n_weights,
    activation=tf.nn.relu,
    kernel_regularizer=tf.keras.regularizers.l2(lamb),
)(char_model3)


In [18]:
from ast import literal_eval
from collections import Counter
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report
from sherlock.deploy.model import SherlockModel

X_train = pd.read_parquet('../data/data/processed/train.parquet')
y_train = pd.read_parquet('../data/data/raw/train_labels.parquet').values.flatten()
y_train = np.array([x.lower() for x in y_train])

X_val = pd.read_parquet('../data/data/processed/validation.parquet')
y_val = pd.read_parquet('../data/data/raw/val_labels.parquet').values.flatten()
y_val = np.array([x.lower() for x in y_val])

X_test = pd.read_parquet('../data/data/processed/test.parquet')
y_test = pd.read_parquet('../data/data/raw/test_labels.parquet').values.flatten()
y_test = np.array([x.lower() for x in y_test])



In [19]:
from sherlock.deploy import helpers
from sklearn.preprocessing import LabelEncoder




encoder = LabelEncoder()
encoder.fit(y_train)

feature_cols = helpers.categorize_features()


X_train_char = X_train[feature_cols["char"]]
X_train_word = X_train[feature_cols["word"]]
X_train_par = X_train[feature_cols["par"]]
X_train_rest = X_train[feature_cols["rest"]]
X_val_char = X_val[feature_cols["char"]]
X_val_word = X_val[feature_cols["word"]]
X_val_par = X_val[feature_cols["par"]]
X_val_rest = X_val[feature_cols["rest"]]

y_train_int = encoder.transform(y_train)
y_val_int = encoder.transform(y_val)
y_train_cat = tf.keras.utils.to_categorical(y_train_int)
y_val_cat = tf.keras.utils.to_categorical(y_val_int)

/mnt/d/GitCode/sherlock-project/sherlock/deploy/helpers.py:18: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  feature_cols_dict[feature_set] = pd.read_csv(
/mnt/d/GitCode/sherlock-project/sherlock/deploy/helpers.py:18: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  feature_cols_dict[feature_set] = pd.read_csv(
/mnt/d/GitCode/sherlock-project/sherlock/deploy/helpers.py:18: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  feature_cols_dict[feature_set] = pd.read_csv(
/mnt/d/GitCode/sherlock-project/sherlock/deploy/helpers.py:18: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  fe

In [20]:
char_model_input #960
char_model1 #960
char_model2 #300
char_model3 #300
char_model4 #300 kerasTensor

<KerasTensor: shape=(None, 300) dtype=float32 (created by layer 'dense_10')>

In [21]:
#WORD
n_weights = 200

word_model_input = Input(shape=(word_shape,))
word_model1 = BatchNormalization(axis=1)(word_model_input)
word_model2 = Dense(
    n_weights,
    activation=tf.nn.relu,
    kernel_regularizer=tf.keras.regularizers.l2(lamb),
)(word_model1)
word_model3 = Dropout(do)(word_model2)
word_model4 = Dense(
    n_weights,
    activation=tf.nn.relu,
    kernel_regularizer=tf.keras.regularizers.l2(lamb),
)(word_model3)

print(word_model_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 201), dtype=tf.float32, name='input_6'), name='input_6', description="created by layer 'input_6'")


In [22]:
#PAR
n_weights = 400

par_model_input = Input(shape=(par_shape,))

print(par_model_input)
print(par_model_input[0])

print(par_model_input[0][0])



par_model1 = BatchNormalization(axis=1)(par_model_input)
par_model2 = Dense(
    n_weights,
    activation=tf.nn.relu,
    kernel_regularizer=tf.keras.regularizers.l2(lamb),
)(par_model1)
par_model3 = Dropout(do)(par_model2)
par_model4 = Dense(
    n_weights,
    activation=tf.nn.relu,
    kernel_regularizer=tf.keras.regularizers.l2(lamb),
)(par_model3)



rest_model_input = Input(shape=(rest_shape,))
rest_model1 = BatchNormalization(axis=1)(rest_model_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 400), dtype=tf.float32, name='input_7'), name='input_7', description="created by layer 'input_7'")
KerasTensor(type_spec=TensorSpec(shape=(400,), dtype=tf.float32, name=None), name='tf.__operators__.getitem_3/strided_slice:0', description="created by layer 'tf.__operators__.getitem_3'")
KerasTensor(type_spec=TensorSpec(shape=(), dtype=tf.float32, name=None), name='tf.__operators__.getitem_5/strided_slice:0', description="created by layer 'tf.__operators__.getitem_5'")


In [23]:
merged_model1 = concatenate([char_model4, word_model4, par_model4, rest_model1])

def _add_main_layers(merged_model1, num_classes):
    n_weights = 500

    merged_model2 = BatchNormalization(axis=1)(merged_model1)
    merged_model3 = Dense(
        n_weights,
        activation=tf.nn.relu,
        kernel_regularizer=tf.keras.regularizers.l2(lamb),
    )(merged_model2)
    merged_model4 = Dropout(do)(merged_model3)
    merged_model5 = Dense(
        n_weights,
        activation=tf.nn.relu,
        kernel_regularizer=tf.keras.regularizers.l2(lamb),
    )(merged_model4)
    merged_model_output = Dense(
        num_classes,
        activation=tf.nn.softmax,
        kernel_regularizer=tf.keras.regularizers.l2(lamb),
    )(merged_model5)

    return merged_model_output
    
    
merged_model_output = _add_main_layers(merged_model1, 78) #tensor

print(merged_model_output)


model = Model(
            [char_model_input, word_model_input, par_model_input, rest_model_input],
            merged_model_output,
        )

model.summary()




KerasTensor(type_spec=TensorSpec(shape=(None, 78), dtype=tf.float32, name=None), name='dense_17/Softmax:0', description="created by layer 'dense_17'")
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 960)]        0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 201)]        0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 

In [24]:
print([char_model_input, word_model_input, par_model_input, rest_model_input])

[<KerasTensor: shape=(None, 960) dtype=float32 (created by layer 'input_5')>, <KerasTensor: shape=(None, 201) dtype=float32 (created by layer 'input_6')>, <KerasTensor: shape=(None, 400) dtype=float32 (created by layer 'input_7')>, <KerasTensor: shape=(None, 27) dtype=float32 (created by layer 'input_8')>]


testing tensorboard

In [25]:
log_dir = "logs"  # local log directory
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [26]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=lr),
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)


/home/sunny/miniconda3/envs/env_jax_3/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [27]:
model.fit(
    [
        X_train_char.values,
        X_train_word.values,
        X_train_par.values,
        X_train_rest.values,
    ],
    y_train_cat,
    validation_data=(
        [
            X_val_char.values,
            X_val_word.values,
            X_val_par.values,
            X_val_rest.values,
        ],
        y_val_cat,
    ),
    callbacks=[tensorboard_callback],
    epochs=1,
    batch_size=256,
)



1610/1610 [==============================] - 38s 22ms/step - loss: 1.6038 - categorical_accuracy: 0.7000 - val_loss: 1.0343 - val_categorical_accuracy: 0.8300


In [28]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 11644), started 4 days, 19:20:08 ago. (Use '!kill 11644' to kill it.)